In [34]:
import pandas as pd
import yfinance as yf
from datetime import datetime

In [35]:
start_date=datetime.now()-pd.DateOffset(months=3)
end_date=datetime.now()

In [36]:
tickers=['AAPL','MSFT','NFLX','GOOG']

In [37]:
df_list=[]

In [38]:
for ticker in tickers:
    data=yf.download(ticker,start=start_date,end=end_date)
    df_list.append(data)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [39]:
df=pd.concat(df_list,keys=tickers,names=['Tickers','Date'])
print(df.head)

<bound method NDFrame.head of                           Open        High         Low       Close  \
Tickers Date                                                         
AAPL    2023-07-31  196.059998  196.490005  195.259995  196.449997   
        2023-08-01  196.240005  196.729996  195.279999  195.610001   
        2023-08-02  195.039993  195.179993  191.850006  192.580002   
        2023-08-03  191.570007  192.369995  190.690002  191.169998   
        2023-08-04  185.520004  187.380005  181.919998  181.990005   
...                        ...         ...         ...         ...   
GOOG    2023-10-24  139.160004  140.710007  138.750000  140.119995   
        2023-10-25  129.770004  130.100006  126.089996  126.669998   
        2023-10-26  124.470001  125.459999  122.320000  123.440002   
        2023-10-27  124.029999  124.440002  121.459999  123.400002   
        2023-10-30  124.459999  126.550003  123.879997  125.750000   

                     Adj Close     Volume  
Tickers Date   

In [40]:
df=df.reset_index()
print(df.head)

<bound method NDFrame.head of     Tickers       Date        Open        High         Low       Close  \
0      AAPL 2023-07-31  196.059998  196.490005  195.259995  196.449997   
1      AAPL 2023-08-01  196.240005  196.729996  195.279999  195.610001   
2      AAPL 2023-08-02  195.039993  195.179993  191.850006  192.580002   
3      AAPL 2023-08-03  191.570007  192.369995  190.690002  191.169998   
4      AAPL 2023-08-04  185.520004  187.380005  181.919998  181.990005   
..      ...        ...         ...         ...         ...         ...   
255    GOOG 2023-10-24  139.160004  140.710007  138.750000  140.119995   
256    GOOG 2023-10-25  129.770004  130.100006  126.089996  126.669998   
257    GOOG 2023-10-26  124.470001  125.459999  122.320000  123.440002   
258    GOOG 2023-10-27  124.029999  124.440002  121.459999  123.400002   
259    GOOG 2023-10-30  124.459999  126.550003  123.879997  125.750000   

      Adj Close     Volume  
0    196.185074   38824100  
1    195.346207   35175

In [49]:
#Now let’s have a look at the performance in the stock market of all the companies
import plotly.express as px
fig = px.line(df, x='Date', 
              y='Close', 
              color='Tickers', 
              title="Stock Market Performance for Apple,Microsoft,Netflix and Google the Last 3 Months")
fig.show()

In [55]:
#Now let’s look at the faceted area chart, which makes it easy to compare the performance of different companies and 
#identify similarities or differences in their stock price movements
fig=px.area(df,x='Date',y='Close', color='Tickers',
           facet_col='Tickers',labels={'Date':'Date','Close':'Closing Price','Ticker':'Company'},
            title='Stock Prices for Apple, Microsoft, Netflix, and Google'
           )
fig.show()

In [61]:
#Now let’s analyze moving averages, which provide a useful way to identify trends and patterns in each company’s stock
#price movements over a period of time:
df['AU10'] = df.groupby('Tickers')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['AU20'] = df.groupby('Tickers')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Tickers'):
    print(f'Moving Averages for {ticker}')
    print(group[['AU10', 'AU20']])

Moving Averages for AAPL
          AU10        AU20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
60  176.585002  175.388502
61  175.715002  175.422002
62  174.333002  175.232002
63  173.270001  175.082502
64  172.427000  174.909502

[65 rows x 2 columns]
Moving Averages for GOOG
           AU10        AU20
195         NaN         NaN
196         NaN         NaN
197         NaN         NaN
198         NaN         NaN
199         NaN         NaN
..          ...         ...
255  139.506999  137.483501
256  138.003999  137.244000
257  136.319000  136.759500
258  134.801000  136.337000
259  133.327000  135.866000

[65 rows x 2 columns]
Moving Averages for MSFT
           AU10        AU20
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
68          NaN         NaN
69          NaN         NaN
..          ...         ...
125  330.396005  325.25

In [64]:
#Now here’s how to visualize the moving averages of all companies:
for ticker, group in df.groupby('Tickers'):
    fig = px.line(group, x='Date', y=['Close', 'AU10', 'AU20'], 
                  title=f"{ticker} Moving Averages")
    fig.show()

In [67]:
#Let us now analyze the volatility of all companies. Volatility is a measure of how much and how often the 
#stock price or market fluctuates over a given period of time. 
#Here’s how to visualize the volatility of all companies
df['Volatility'] = df.groupby('Tickers')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility', 
              color='Tickers', 
              title='Volatility of All Companies')
fig.show()

In [69]:
#Now let’s analyze the correlation between the stock prices of Apple and Microsoft:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Tickers'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Tickers'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT', 
                 trendline='ols', 
                 title='Correlation between Apple and Microsoft')
fig.show()